In [1]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine
from config import connect as conn

In [2]:
# Create Engine and connection
engine = create_engine(f"postgresql://{conn['user']}:{conn['password']}@{conn['host']}/{conn['database']}")
connection = engine.connect()

In [3]:
# Q1: List the employee number, last name, first name, sex, and salary of each employee.
data = pd.read_sql('''SELECT employee.emp_no, last_name, first_name, sex, salary 
                      FROM employee 
                      INNER JOIN salary 
                      ON employee.emp_no = salary.emp_no;''', connection)
data.head()

,emp_no,last_name,first_name,sex,salary
0,57444,Babb,Berry,F,48973
1,263976,Cusworth,Eran,M,40000
2,461591,Samarati,Xudong,M,40000
3,477657,Magliocco,Lihong,M,54816
4,29920,Tyugu,Shuichi,F,40000


In [4]:
# Q2: List the first name, last name, and hire date for the employees who were hired in 1986.
data = pd.read_sql('''SELECT first_name, last_name, hire_date 
                      FROM employee 
                      WHERE extract(year from hire_date) = 1986;''', connection)
data.head()

,first_name,last_name,hire_date
0,Eran,Cusworth,1986-11-14
1,Bojan,Zallocco,1986-10-14
2,Nevio,Demizu,1986-05-18
3,Ziva,Vecchi,1986-07-03
4,Mohit,Speek,1986-01-14


In [5]:
# Q3: List the manager of each department along with their department number, department name, employee number,
#     last name, and first name using SUBQUERY and JOIN
data = pd.read_sql('''SELECT dept_no, dept_name, employee.emp_no, last_name, first_name
                      FROM employee
                      INNER JOIN (
                          SELECT dept_no, 
                                 (SELECT dept_name
                                 FROM department
                                 WHERE dept_manager.dept_no = department.dept_no), 
                                 emp_no
                          FROM dept_manager
                      ) As dept
                      ON employee.emp_no = dept.emp_no;''', connection)
data.head()

,dept_no,dept_name,emp_no,last_name,first_name
0,d001,Marketing,110022,Markovitch,Margareta
1,d001,Marketing,110039,Minakawa,Vishwani
2,d002,Finance,110085,Alpin,Ebru
3,d002,Finance,110114,Legleitner,Isamu
4,d003,Human Resources,110183,Ossenbruggen,Shirish


In [6]:
# Q4: List the department number for each employee along with that employee's employee number, last name,
#     first name, and department name using JOINS
data = pd.read_sql('''SELECT department.dept_no, employee.emp_no, last_name, first_name, dept_name
                      FROM (dept_employee
                            INNER JOIN employee
                            ON dept_employee.emp_no = employee.emp_no
                      ) INNER JOIN department
                      ON dept_employee.dept_no = department.dept_no;''', connection)
data.head()

,dept_no,emp_no,last_name,first_name,dept_name
0,d005,10001,Facello,Georgi,Development
1,d003,10005,Maliniak,Kyoichi,Human Resources
2,d004,10010,Piveteau,Duangkaew,Production
3,d006,10010,Piveteau,Duangkaew,Quality Management
4,d009,10011,Sluis,Mary,Customer Service


In [7]:
# Q5: List first name, last name, and sex of each employee whose first name is Hercules and whose last name
#     begins with the letter B. (need to escape % using %%)
data = pd.read_sql("""SELECT first_name, last_name, sex
                      FROM employee
                      WHERE first_name = 'Hercules'
                      And last_name LIKE 'B%%';""", connection)
data.head()

,first_name,last_name,sex
0,Hercules,Baer,M
1,Hercules,Biron,F
2,Hercules,Birge,F
3,Hercules,Berstel,F
4,Hercules,Bernatsky,M


In [8]:
# Q6: List each employee in the Sales department, including their employee number, last name, and first name
data = pd.read_sql("""SELECT dept_name, employee.emp_no, last_name, first_name
                      FROM (dept_employee
                            INNER JOIN employee
                            ON dept_employee.emp_no = employee.emp_no
                      ) INNER JOIN department
                      ON dept_employee.dept_no = department.dept_no
                      WHERE dept_name = 'Sales';""", connection)
data.head()

,dept_name,emp_no,last_name,first_name
0,Sales,10002,Simmel,Bezalel
1,Sales,10016,Cappelletti,Kazuhito
2,Sales,10034,Swan,Bader
3,Sales,10041,Lenart,Uri
4,Sales,10050,Dredge,Yinghua


In [9]:
# Q7: List each employee in the Sales and Development departments, including their employee number, last name,
#     first name, and department name.
data = pd.read_sql("""SELECT dept_name, employee.emp_no, last_name, first_name
                      FROM (dept_employee
                            INNER JOIN employee
                            ON dept_employee.emp_no = employee.emp_no
                      ) INNER JOIN department
                      ON dept_employee.dept_no = department.dept_no
                      WHERE dept_name IN ('Sales', 'Development');""", connection)
data.head()

,dept_name,emp_no,last_name,first_name
0,Development,10001,Facello,Georgi
1,Sales,10002,Simmel,Bezalel
2,Development,10006,Preusig,Anneke
3,Development,10008,Kalloufi,Saniya
4,Development,10012,Bridgland,Patricio


In [10]:
# Q8: List the frequency counts, in descending order, of all the employee last names (that is, how many employees
#     share each last name).
data = pd.read_sql('''SELECT last_name, Count(last_name) AS "frequency"
                      FROM employee
                      GROUP BY last_name
                      ORDER BY frequency DESC;''', connection)
data.head()

,last_name,frequency
0,Baba,226
1,Coorg,223
2,Gelosh,223
3,Sudbeck,222
4,Farris,222


In [11]:
connection.close
engine.dispose()